<a href="https://colab.research.google.com/github/Jegovila/cursoVR/blob/main/Practica6%3A%20Segmentaci%C3%B3n%20por%20color/python/Pr%C3%A1ctica6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Captura de imagen

In [1]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

# Recorte de la imagen

In [4]:
import cv2
from matplotlib import pyplot as plt

In [ ]:
I = cv2.imread('photo.jpg')
#I = cv2.resize(I, (500, 333))
#plt.axis("off")
plt.imshow(cv2.cvtColor(I, cv2.COLOR_BGR2RGB))
print(I.shape)

In [ ]:
x=100 # int(input("Enter Start point on x axis:"))
y=250 # int(input("Enter Start point on y axis:"))

w=100 # int(input("Enter width of cropped image:"))
h=50  # int(input("Enter height of cropped image:"))

cropped_image = I[y:y+h, x:x+w] # Cropping using Slicing
plt.axis("off")
plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
plt.show() # Display the original image
print(cropped_image.shape)

cv2.imwrite("crop.jpg", cropped_image)

# Segmentación por color

In [16]:
import numpy as np

In [22]:
I = cv2.imread("photo.jpg")
Iroi = cv2.imread("crop.jpg")
b, g, r = cv2.split(Iroi)
m, n, o = I.shape

# Media
# Ar = np.mean(Iroi[:,:,2])    # recordar que es -> BGR
Ar = np.mean(r)
Ag = np.mean(g)
Ab = np.mean(b)

# Convertir a double
Rd = r.astype(np.float64)
Gd = g.astype(np.float64)
Bd = b.astype(np.float64)

# std
Dr = np.std(Rd)
Dg = np.std(Gd)
Db = np.std(Bd)

Detección

In [23]:
Ig = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
IBW = np.zeros((m, n), dtype=np.uint8)

s = 2

for i in range(m):
    for j in range(n):
        if ((Ar + s * Dr) > I[i, j, 2] > (Ar - s * Dr) and
                (Ag + s * Dg) > I[i, j, 1] > (Ag - s * Dg) and
                (Ab + s * Db) > I[i, j, 0] > (Ab - s * Db)):
            IBW[i, j] = 255
        else:
            IBW[i, j] = 0


Encerrar los blobs que sean True

In [24]:
try:
    white_pixels = np.array(np.where(IBW == 255))
    min_fila = np.min(white_pixels[0, :])
    min_col = np.min(white_pixels[1, :])
    max_fila = np.max(white_pixels[0, :])
    max_col = np.max(white_pixels[1, :])
    start = [min_col, min_fila]
    end = [max_col, max_fila]
    # image = cv2.rectangle(image, start_point, end_point, color, thickness)
    I = cv2.rectangle(I, start, end, (0, 255, 0), 2)

    center2 = min_fila + (max_fila - min_fila) / 2
    center1 = min_col + (max_col - min_col) / 2
    # image = cv.circle(image, centerOfCircle, radius, color, thickness)
    I = cv2.circle(I, [int(center1), int(center2)], 10, (0, 255, 255), -1)
except:
    pass

Resultados

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,14))
fig.tight_layout()

plt.subplot(1,2,1)
plt.imshow(cv2.cvtColor(I, cv2.COLOR_BGR2RGB))
plt.subplot(1,2,2)
plt.imshow(255-IBW, cmap="binary")